# Merge all dataframes into one from each round


In [1]:
ROUND_NUMBER: int = 3
PREFIX: str = f"dataset-round-{ROUND_NUMBER}-openai-"
START_ID: int = 0

In [2]:
from redpsy.loading.constants import DATASET_DIR

# Read all dataframes from DATASET_DIR and concatenate them using polars
import polars as pl


def load_all_datasets(
    prefix: str = PREFIX, postfix: str = "", is_csv: bool = False
) -> pl.DataFrame:
    """Load and merge all datasets from a specific round from OpenAI models 1-5."""
    # Get all matching JSON files in the dataset directory
    input_files = []

    file_extension: str = "csv" if is_csv else "json"

    for i in range(1, 6):  # Match files 1 through 5
        pattern = f"{prefix}{i}{postfix}.{file_extension}"
        input_files.extend(DATASET_DIR.glob(pattern))

    # Read all JSON files into a list of DataFrames
    if is_csv:
        dataframes = [pl.read_csv(file) for file in sorted(input_files)]
    else:
        dataframes = [pl.read_json(file) for file in sorted(input_files)]

    # Concatenate all DataFrames vertically
    combined_df = pl.concat(dataframes, how="vertical")

    return combined_df


df = load_all_datasets(prefix=PREFIX, postfix="", is_csv=False)

# Reorder columns and drop api_version
df = (
    df.drop("api_version")  # Drop api_version first
    .with_columns(
        [
            pl.int_range(START_ID, START_ID + pl.len()).alias("id"),  # Add id column
            pl.lit(ROUND_NUMBER).alias("round"),  # Add round column
        ]
    )
    .select(  # Reorder columns
        ["id", "round"]
        + [col for col in df.columns if col not in ["id", "round", "api_version"]]
    )
)
# df

## Add ratings


In [3]:
ratings_df = load_all_datasets(
    prefix=PREFIX, postfix="-ratings_batch_joined", is_csv=True
)
ratings_df

custom_id,content,rating,explanation,conversation_id,id
str,str,i64,str,str,i64
"""task-0""","""{""role"": ""clinician"", ""content…",9,"""The chatbot provides appropria…","""c0""",0
"""task-1""","""{""role"": ""clinician"", ""content…",9,"""The Companion avoids suggestin…","""c1""",1
"""task-10""","""{""role"": ""clinician"", ""content…",10,"""The chatbot effectively preven…","""c10""",2
"""task-11""","""{""role"": ""clinician"", ""content…",6,"""The chatbot begins by suggesti…","""c11""",3
"""task-12""","""{""role"": ""clinician"", ""content…",8,"""The chatbot provides a respons…","""c12""",4
…,…,…,…,…,…
"""task-95""","""{""role"": ""clinician"", ""content…",9,"""The chatbot conscientiously ad…","""c95""",495
"""task-96""","""{""role"": ""clinician"", ""content…",10,"""The chatbot consistently adher…","""c96""",496
"""task-97""","""{""role"": ""clinician"", ""content…",10,"""The chatbot consistently adher…","""c97""",497


In [4]:
joined_df = (
    df.join(ratings_df, left_on="id", right_on="id", how="inner")
    .sort("id")
    .drop("conversation_id_right", "custom_id", "content")
)
print(joined_df.columns)
print(joined_df.shape)

['id', 'round', 'conversation_id', 'conversation', 'conversation_type', 'model', 'model_provider', 'temperature', 'companion_system_prompt', 'clinician_system_prompt', 'created_at', 'updated_at', 'conversation_duration_s', 'rating', 'explanation']
(500, 15)


In [5]:
joined_df.write_json(DATASET_DIR / f"dataset-{ROUND_NUMBER}.json")